In [1]:
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace,Dataset
from random import randrange,randint
from sklearn.externals import joblib
import sys

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
ws = Workspace.from_config()

In [3]:
max_depth = randint(2,10)
n_estimators = int(randrange(2000,5000,100))

In [4]:
experiment = Experiment(ws, 'gbr-turbofan')

dataset = Dataset.get_by_name(workspace, name='NASA TurboFan Raw')
train = dataset.to_pandas_dataframe()

X = train.drop('rul',axis=1)
y = pd.Series(train.rul)


run =  experiment.start_logging()
run.tag("python version", sys.version[0:6])

 # Log the algorithm parameter alpha to the run
run.log('max_depth', max_depth)
run.log('n_estimators', n_estimators)


X_train, X_test, y_train, y_test = train_test_split(X, y)


regression_model = GradientBoostingRegressor(
    max_depth=max_depth,
    n_estimators=n_estimators,
    learning_rate=.5
)

regression_model.fit(X_train, y_train)

y_pred = regression_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
run.log('mae', mae)

# Save the model to the outputs directory for capture
model_file_name = 'outputs/model.pkl'

joblib.dump(value = regression_model, filename = model_file_name)

# upload the model file explicitly into artifacts 
#run.upload_file(name = model_file_name, path_or_stream = model_file_name)

# Complete the run
run.complete()